### Demo Ordini di Servizio Sogei with Cohere


* see: https://python.langchain.com/docs/use_cases/question_answering/

In [1]:
# Document Loader
from langchain.document_loaders import DirectoryLoader

# Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Embeddings and Vector Store (Vector DB)
# removed OpenAI, using HF for embeddings
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

from langchain import hub

# removed OpenAI, using Cohere
from langchain.chat_models import ChatCohere

from langchain.schema.runnable import RunnablePassthrough

# private configurations
from config_private import (COHERE_API_KEY)

#### Loading the document: a set of local txt documents

In [2]:
LOCAL_DIR = "./docs"
LOCAL_EXT = "*.txt"

loader = DirectoryLoader(LOCAL_DIR, glob=LOCAL_EXT)

data = loader.load()

In [3]:
print(f"We have loaded {len(data)} documents...")

We have loaded 5 documents...


#### Splitting the document in chunks

In [4]:
CHUNK_SIZE = 1024

text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=0)

splits = text_splitter.split_documents(data)

In [5]:
print(f"We have {len(splits)} splits...")

We have 18 splits...


In [6]:
splits[1]

Document(page_content='quale riportano le seguenti strutture: \uf0b7 Unità Organizzativa Servizi per il Personale PA Responsabile: Barbara Paci; Pag.', metadata={'source': 'docs/OdS_2020-4.txt'})

#### Create Vector DB and embeddings

In [7]:
# We have substituted OpenAI with HF
EMBED_MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"

model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}


hf = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL_NAME, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# using Chroma as Vector store
vectorstore = Chroma.from_documents(documents=splits, embedding=hf)

retriever = vectorstore.as_retriever()

#### Define the prompt structure

In [8]:
rag_prompt = hub.pull("rlm/rag-prompt")

#### Define the LLM

In [9]:
API_KEY = COHERE_API_KEY

LLM_NAME = "command-nightly"

llm = ChatCohere(cohere_api_key=API_KEY, temperature=0.5, verbose=True, model=LLM_NAME)

#### Define the (Lang)Chain

In [10]:
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | rag_prompt | llm

#### Process the question

In [11]:
QUESTION = (
    """Chi è il responsabile della Funzione RGS Sistemi ERP? Rispondi in italiano"""
)

In [12]:
response = rag_chain.invoke(QUESTION)

print(response.content)

Gianna Caralla


#### Explore the vectore store

In [13]:
# Retrieve relevant splits for any question using similarity search.

# This is simply "top K" retrieval where we select documents based on embedding similarity to the query.

TOP_K = 5

docs = vectorstore.similarity_search(QUESTION, k=TOP_K)

len(docs)

5

In [14]:
for i in range(len(docs)):
    print(f"docs[{i}]:")
    print(docs[i].page_content)

docs[0]:
relazione con RGS, così come previsto nel relativo nuovo disciplinare. Andrea Quacivi Amministratore Delegato Firmato digitalmente TERMINE DI PUBBLICAZIONE: 23 luglio 2020
docs[1]:
La Funzione si articola nelle seguenti strutture:  Unità Organizzativa Plants Design & Service Development Responsabile: Paolo Piacente;  Unità Organizzativa Operation & Maintenance Responsabile: Paolo Franzese; La Direzione Cloud Data Center, la cui responsabilità è affidata a Luca Bargellini, cura lo sviluppo e l’evoluzione dell’infrastruttura tecnologica e di sicurezza per la realizzazione e l’erogazione dei servizi e delle soluzioni IT a supporto degli ecosistemi di business. La Direzione si articola in:  Funzione Change & Service Management, affidata a Giuseppe Spoto, assicura le attività di governo di change, release e configuration management. Pag. 3 di 5 La Funzione si articola nelle seguenti strutture:  Unità Organizzativa Change & Release Management Responsabile: Francesco Milone;  Un